# Generating data files (.pt)
#### Setup instructions
Ensure that FeatureData_FakeMatl exists in order to generate graph.

In [2]:
import import_ipynb
from Visualize_Microstructures import gen_graph, DIR_LOC

import os
import pathlib

import numpy as np
from torch_geometric.utils import from_networkx
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader

GEN_STRUCTURES_FILE_BASE = os.path.join(DIR_LOC, "generated_microstructures", "FeatureData_FakeMatl_")
NUM_MICROSTRUCTURES = 99

importing Jupyter notebook from Visualize_Microstructures.ipynb


In [11]:
def network_to_pyg_data(file):
    G = gen_graph(file)
    pyg_graph = from_networkx(G, group_node_attrs=["pos"], group_edge_attrs=["weight"])
    pyg_graph.y = pyg_graph["surfaceFeature"]
    del pyg_graph["surfaceFeature"]
    pyg_graph.y = pyg_graph.y.type(torch.LongTensor)

    # Split the data
    train_ratio = 0.2
    num_nodes = pyg_graph.x.shape[0]
    num_train = int(num_nodes * train_ratio)
    idx = [i for i in range(num_nodes)]

    np.random.shuffle(idx)
    train_mask = torch.full_like(pyg_graph.y, False, dtype=bool)
    train_mask[idx[:num_train]] = True
    test_mask = torch.full_like(pyg_graph.y, False, dtype=bool)
    test_mask[idx[num_train:]] = True

    data = pyg_graph
    data.train_mask = train_mask
    data.test_mask = test_mask

    return data

In [15]:
if __name__ == "__main__":
    data_batch = []
    for i in range(0, NUM_MICROSTRUCTURES):
        file = GEN_STRUCTURES_FILE_BASE + str(i) + ".csv"
        print("Loading graph " + str(i) + "...")
        data_batch.append(network_to_pyg_data(file))
    
    print(data_batch)
    # loader to combine data
    print("Combining data...")
    
    loader = DataLoader(data_batch, batch_size=32)
    data = next(iter(loader))
    data

Loading graph 0...
Loading graph 1...
Loading graph 2...
Loading graph 3...
Loading graph 4...
Loading graph 5...
Loading graph 6...
Loading graph 7...
Loading graph 8...
Loading graph 9...
Loading graph 10...
Loading graph 11...
Loading graph 12...
Loading graph 13...
Loading graph 14...
Loading graph 15...
Loading graph 16...
Loading graph 17...
Loading graph 18...
Loading graph 19...
Loading graph 20...
Loading graph 21...
Loading graph 22...
Loading graph 23...
Loading graph 24...
Loading graph 25...
Loading graph 26...
Loading graph 27...
Loading graph 28...
Loading graph 29...
Loading graph 30...
Loading graph 31...
Loading graph 32...
Loading graph 33...
Loading graph 34...
Loading graph 35...
Loading graph 36...
Loading graph 37...
Loading graph 38...
Loading graph 39...
Loading graph 40...
Loading graph 41...
Loading graph 42...
Loading graph 43...
Loading graph 44...
Loading graph 45...
Loading graph 46...
Loading graph 47...
Loading graph 48...
Loading graph 49...
Loading gr

In [16]:
torch.save(data, "datasets/data.pt")

In [17]:
data

DataBatch(edge_index=[2, 109830], rot=[7942, 3], size=[7942], x=[7942, 3], edge_attr=[109830, 1], y=[7942], train_mask=[7942], test_mask=[7942], batch=[7942], ptr=[33])